In [15]:
! py -m pip list

Package                                  Version
---------------------------------------- -------------------
absl-py                                  1.0.0
aiohttp                                  3.9.5
aiosignal                                1.3.1
altair                                   5.3.0
annotated-types                          0.6.0
anyio                                    3.5.0
argon2-cffi                              21.3.0
argon2-cffi-bindings                     21.2.0
asgiref                                  3.8.1
asttokens                                2.0.5
astunparse                               1.6.3
async-timeout                            4.0.3
asyncio                                  3.4.3
attrs                                    21.4.0
Babel                                    2.9.1
backcall                                 0.2.0
backoff                                  2.2.1
bcrypt                                   4.1.3
beautifulsoup4                           

In [5]:
import pandas as pd
import sqlite3
import random
from datetime import datetime, timedelta
from faker import Faker
import sys

# Initialize Faker for generating fake names.
fake = Faker()

# Helper function to generate a random timestamp between two datetime objects.
def random_timestamp(start, end):
    delta = end - start
    random_seconds = random.randint(0, int(delta.total_seconds()))
    return start + timedelta(seconds=random_seconds)

# Helper function to generate a random date between two dates.
def random_date(start, end):
    delta = end - start
    random_days = random.randint(0, delta.days)
    return start + timedelta(days=random_days)

# Define ingestion timestamp range (for RecordIngestedOn)
ingest_start = datetime(2025, 1, 1)
ingest_end = datetime(2025, 3, 31)

# Define date of birth range: residents will be at least 65 in 2025 if born on or before 1960.
dob_start = datetime(1940, 1, 1)
dob_end = datetime(1960, 12, 31)

num_residents = 500
data = []

for i in range(1, num_residents + 1):
    resident_name = fake.name()
    # Generate a DOB between 1900 and 1960, then format as M/D/YYYY (or MM/DD/YYYY)
    if sys.platform.startswith("win"):
        dob_format = "%#m/%#d/%Y"
    else:
        dob_format = "%-m/%-d/%Y"

    dob = random_date(dob_start, dob_end).strftime(dob_format)
    # For Windows, you might use: dob = random_date(dob_start, dob_end).strftime("%#m/%#d/%Y")
    record_ingested_on = random_timestamp(ingest_start, ingest_end).strftime("%Y-%m-%d %H:%M:%S")
    
    row = {
        "ResidentKey": i,
        "ResidentName": resident_name,
        "ResidentDateOfBirth": dob,
        "RecordIngestedOn": record_ingested_on
    }
    data.append(row)

# Create a DataFrame from the data.
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file.
csv_filename = "Dim_Resident.csv"
df.to_csv(csv_filename, index=False)
print(f"CSV file '{csv_filename}' created with {num_residents} records.")

# Connect to the SQLite database (medical_events.db) and store the data in the Dim_Resident table.
db_filename = "medical_events.db"
conn = sqlite3.connect(db_filename)
df.to_sql("Dim_Resident", conn, if_exists='replace', index=False)
conn.commit()
conn.close()
print(f"Data stored in SQLite database '{db_filename}' in table 'Dim_Resident'.")


CSV file 'Dim_Resident.csv' created with 500 records.
Data stored in SQLite database 'medical_events.db' in table 'Dim_Resident'.


In [24]:
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import random

# List of county names for each branch.
counties = [
    "Parker at Somerset",
    "Parker at River Road",
    "Parker at Monroe",
    "Parker at Landing Lane",
    "Parker at Stonegate"
]

# Function to generate a random timestamp between two datetime objects.
def random_timestamp(start, end):
    delta = end - start
    random_seconds = random.randint(0, int(delta.total_seconds()))
    return start + timedelta(seconds=random_seconds)

# Set the date range for RecordIngestedOn.
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 3, 31)

# Create the data for each facility using only county names.
data = []
for i, county in enumerate(counties, start=1):
    facility_code = f"{i:02d}"  # Two-digit facility code, e.g., "01", "02", etc.
    facility_name = f"Reliant care at {county}"
    record_ingested_on = random_timestamp(start_date, end_date).strftime("%Y-%m-%d %H:%M:%S")
    row = {
        "FacilityKey": i,
        "FacilityCode": facility_code,
        "FacilityName": facility_name,
        "RecordIngestedOn": record_ingested_on
    }
    data.append(row)

# Convert the list of dictionaries into a DataFrame.
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file.
csv_filename = "Dim_Facility.csv"
df.to_csv(csv_filename, index=False)
print(f"CSV file '{csv_filename}' created with {len(df)} records.")

# Create (or connect to) the SQLite database and store the data in the Dim_CensusFacility table.
db_filename = "medical_events.db"
conn = sqlite3.connect(db_filename)
df.to_sql("Dim_Facility", conn, if_exists='replace', index=False)
conn.commit()
conn.close()
print(f"Data stored in SQLite database '{db_filename}' in table 'Dim_Facility'.")


CSV file 'Dim_Facility.csv' created with 5 records.
Data stored in SQLite database 'medical_events.db' in table 'Dim_Facility'.


In [23]:
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import random

# List of county names (from the previous facility example)
counties = [
    "Parker at Somerset",
    "Parker at River Road",
    "Parker at Monroe",
    "Parker at Landing Lane",
    "Parker at Stonegate"
]

# Function to generate a random timestamp between two datetime objects.
def random_timestamp(start, end):
    delta = end - start
    random_seconds = random.randint(0, int(delta.total_seconds()))
    return start + timedelta(seconds=random_seconds)

# Set the date range for RecordIngestedOn.
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 3, 31)

# Create data for each location.
data = []
for i, county in enumerate(counties, start=1):
    row = {
        "LocationKey": i,
        "LocationId": str(i),
        "LocationCode": str(i),  # Using the row number as a string for the code.
        "LocationName": county,
        "RecordIngestedOn": random_timestamp(start_date, end_date).strftime("%Y-%m-%d %H:%M:%S")
    }
    data.append(row)

# Convert the list of dictionaries into a DataFrame.
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file.
csv_filename = "Dim_EHRLocation.csv"
df.to_csv(csv_filename, index=False)
print(f"CSV file '{csv_filename}' created with {len(df)} records.")

# Create (or connect to) the SQLite database and store the data in the Dim_CensusLocation table.
db_filename = "medical_events.db"
conn = sqlite3.connect(db_filename)
df.to_sql("Dim_EHRLocation", conn, if_exists='replace', index=False)
conn.commit()
conn.close()
print(f"Data stored in SQLite database '{db_filename}' in table 'Dim_EHRLocation'.")


CSV file 'Dim_EHRLocation.csv' created with 5 records.
Data stored in SQLite database 'medical_events.db' in table 'Dim_EHRLocation'.


In [22]:
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import random

# Define 10 realistic unit names for senior living.
unit_names = [
"AppleBlossom",
"Red Dogwood",
"Resident Services",
"River Birch",
"River View Way",
"Rose Willow Way",
"SILVER BR",
"Silver Birch",
"Silver Oak Court",
"Spruce Run"
]

# Function to generate a random timestamp between two datetime objects.
def random_timestamp(start, end):
    delta = end - start
    random_seconds = random.randint(0, int(delta.total_seconds()))
    return start + timedelta(seconds=random_seconds)

# Set the date range for RecordIngestedOn.
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 3, 31)

# Create data for each unit.
data = []
for i, unit_name in enumerate(unit_names, start=1):
    row = {
        "UnitKey": i,
        "UnitId": str(i),
        "UnitCode": str(i),  # Using the unit number as the code.
        "UnitName": unit_name,
        "RecordIngestedOn": random_timestamp(start_date, end_date).strftime("%Y-%m-%d %H:%M:%S")
    }
    data.append(row)

# Convert the list of dictionaries into a DataFrame.
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file.
csv_filename = "Dim_Unit.csv"
df.to_csv(csv_filename, index=False)
print(f"CSV file '{csv_filename}' created with {len(df)} records.")

# Create (or connect to) the SQLite database and store the data in the Dim_CensusUnit table.
db_filename = "medical_events.db"
conn = sqlite3.connect(db_filename)
df.to_sql("Dim_Unit", conn, if_exists='replace', index=False)
conn.commit()
conn.close()
print(f"Data stored in SQLite database '{db_filename}' in table 'Dim_Unit'.")


CSV file 'Dim_Unit.csv' created with 10 records.
Data stored in SQLite database 'medical_events.db' in table 'Dim_Unit'.


In [9]:
import pandas as pd
import sqlite3

# List to hold our event type records.
data = []
key = 1

# Group: Infection (4 events)
infection_events = ["Bacterial Infection", "Viral Infection", "Fungal Infection", "Urinary Tract Infection"]
for event in infection_events:
    data.append({
        "MedicalEventTypeKey": key,
        "MedicalEventTypeName": event,
        "MedicalEventTypeGroup": "Infection"
    })
    key += 1

# Group: Medication Error (4 events)
med_error_events = ["Wrong Dosage", "Missed Dose", "Incorrect Medication", "Administration Error"]
for event in med_error_events:
    data.append({
        "MedicalEventTypeKey": key,
        "MedicalEventTypeName": event,
        "MedicalEventTypeGroup": "Medication Error"
    })
    key += 1

# Group: Fall (3 events)
fall_events = ["Slip and Fall", "Trip Fall", "Fainting Fall"]
for event in fall_events:
    data.append({
        "MedicalEventTypeKey": key,
        "MedicalEventTypeName": event,
        "MedicalEventTypeGroup": "Fall"
    })
    key += 1

# Group: Wound (3 events)
wound_events = ["Pressure Ulcer", "Laceration", "Abrasion"]
for event in wound_events:
    data.append({
        "MedicalEventTypeKey": key,
        "MedicalEventTypeName": event,
        "MedicalEventTypeGroup": "Wound"
    })
    key += 1

# Group: Lab (3 events)
lab_events = ["Abnormal Blood Test", "High Cholesterol", "Low Hemoglobin"]
for event in lab_events:
    data.append({
        "MedicalEventTypeKey": key,
        "MedicalEventTypeName": event,
        "MedicalEventTypeGroup": "Lab"
    })
    key += 1

# Group: Behavioral (3 events)
behavioral_events = ["Agitation", "Wandering", "Verbal Aggression"]
for event in behavioral_events:
    data.append({
        "MedicalEventTypeKey": key,
        "MedicalEventTypeName": event,
        "MedicalEventTypeGroup": "Behavioral"
    })
    key += 1

# Group: Equipment Failure (3 events)
equipment_events = ["Ventilator Malfunction", "IV Pump Failure", "Monitoring Device Error"]
for event in equipment_events:
    data.append({
        "MedicalEventTypeKey": key,
        "MedicalEventTypeName": event,
        "MedicalEventTypeGroup": "Equipment Failure"
    })
    key += 1

# Group: Allergy (3 events)
allergy_events = ["Food Allergy Reaction", "Medication Allergy Reaction", "Environmental Allergy Reaction"]
for event in allergy_events:
    data.append({
        "MedicalEventTypeKey": key,
        "MedicalEventTypeName": event,
        "MedicalEventTypeGroup": "Allergy"
    })
    key += 1

# Convert the list of dictionaries to a DataFrame.
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file.
csv_filename = "Dim_MedicalEventType.csv"
df.to_csv(csv_filename, index=False)
print(f"CSV file '{csv_filename}' created with {len(df)} records.")

# Save the data into a SQLite database.
db_filename = "medical_events.db"
conn = sqlite3.connect(db_filename)
df.to_sql("Dim_MedicalEventType", conn, if_exists='replace', index=False)
conn.commit()
conn.close()
print(f"Data stored in SQLite database '{db_filename}' in table 'Dim_MedicalEventType'.")


CSV file 'Dim_MedicalEventType.csv' created with 26 records.
Data stored in SQLite database 'medical_events.db' in table 'Dim_MedicalEventType'.


In [10]:
import pandas as pd
import sqlite3

# Define a list of 20 representative medical event severity values.
# Each severity has a unique key starting from 1.
severity_data = [
    {"MedicalEventSeverityKey": 1, "MedicalEventSeverity": "BLOOD CULTURE #1"},
    {"MedicalEventSeverityKey": 2, "MedicalEventSeverity": "BLOOD CULTURE #2"},
    {"MedicalEventSeverityKey": 3, "MedicalEventSeverity": "Bacteremia"},
    {"MedicalEventSeverityKey": 4, "MedicalEventSeverity": "Bronchitis"},
    {"MedicalEventSeverityKey": 5, "MedicalEventSeverity": "CULTURE, ANAEROBIC"},
    {"MedicalEventSeverityKey": 6, "MedicalEventSeverity": "CULTURE, SPUTUM"},
    {"MedicalEventSeverityKey": 7, "MedicalEventSeverity": "Candidiasis"},
    {"MedicalEventSeverityKey": 8, "MedicalEventSeverity": "Cellulitis"},
    {"MedicalEventSeverityKey": 9, "MedicalEventSeverity": "Covid 19"},
    {"MedicalEventSeverityKey": 10, "MedicalEventSeverity": "Herpes Simplex"},
    {"MedicalEventSeverityKey": 11, "MedicalEventSeverity": "Impetigo"},
    {"MedicalEventSeverityKey": 12, "MedicalEventSeverity": "Influenza A"},
    {"MedicalEventSeverityKey": 13, "MedicalEventSeverity": "Osteomyelitis"},
    {"MedicalEventSeverityKey": 14, "MedicalEventSeverity": "Otitis Media"},
    {"MedicalEventSeverityKey": 15, "MedicalEventSeverity": "Pneumonia"},
    {"MedicalEventSeverityKey": 16, "MedicalEventSeverity": "Sepsis, unspecified"},
    {"MedicalEventSeverityKey": 17, "MedicalEventSeverity": "Septicemia"},
    {"MedicalEventSeverityKey": 18, "MedicalEventSeverity": "Severity Level 1-No Harm/Damage"},
    {"MedicalEventSeverityKey": 19, "MedicalEventSeverity": "Severity Level 3-Serious Injury/Damage"},
    {"MedicalEventSeverityKey": 20, "MedicalEventSeverity": "Severity Level 4-Death"}
]

# Convert the list of dictionaries into a DataFrame.
df = pd.DataFrame(severity_data)

# Save the DataFrame to a CSV file.
csv_filename = "Dim_MedicalEventSeverity.csv"
df.to_csv(csv_filename, index=False)
print(f"CSV file '{csv_filename}' created with {len(df)} records.")

# Connect to the SQLite database and store the data in the Dim_MedicalEventSeverity table.
db_filename = "medical_events.db"
conn = sqlite3.connect(db_filename)
df.to_sql("Dim_MedicalEventSeverity", conn, if_exists='replace', index=False)
conn.commit()
conn.close()
print(f"Data stored in SQLite database '{db_filename}' in table 'Dim_MedicalEventSeverity'.")



CSV file 'Dim_MedicalEventSeverity.csv' created with 20 records.
Data stored in SQLite database 'medical_events.db' in table 'Dim_MedicalEventSeverity'.


In [11]:
import pandas as pd
import sqlite3

# Create a list of dictionaries for event status data.
data = [
    {"EventStatusKey": 1, "EventStatusName": "Closed", "EventStatusReason": "Criteria Not Met"},
    {"EventStatusKey": 2, "EventStatusName": "Closed", "EventStatusReason": "Deceased (In House)"},
    {"EventStatusKey": 3, "EventStatusName": "Closed", "EventStatusReason": "Discharged"},
    {"EventStatusKey": 4, "EventStatusName": "Closed", "EventStatusReason": "Duplicate"},
    {"EventStatusKey": 5, "EventStatusName": "Confirmed (D)", "EventStatusReason": "Error"},
    {"EventStatusKey": 6, "EventStatusName": "Confirmed (P)", "EventStatusReason": "Resolved"},
    {"EventStatusKey": 7, "EventStatusName": "Suspected", "EventStatusReason": "Criteria Not Met"},
    {"EventStatusKey": 8, "EventStatusName": "Suspected", "EventStatusReason": "Error"}
]

# Convert the list of dictionaries into a DataFrame.
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file.
csv_filename = "Dim_EventStatus.csv"
df.to_csv(csv_filename, index=False)
print(f"CSV file '{csv_filename}' created with {len(df)} records.")

# Connect to the SQLite database and store the data in the Dim_EventStatus table.
db_filename = "medical_events.db"
conn = sqlite3.connect(db_filename)
df.to_sql("Dim_EventStatus", conn, if_exists='replace', index=False)
conn.commit()
conn.close()
print(f"Data stored in SQLite database '{db_filename}' in table 'Dim_EventStatus'.")


CSV file 'Dim_EventStatus.csv' created with 8 records.
Data stored in SQLite database 'medical_events.db' in table 'Dim_EventStatus'.


In [12]:
import pandas as pd
import sqlite3
import calendar
from datetime import datetime, timedelta

# Define the date range: from 2021-01-01 to 2024-12-31
start_date = datetime(2021, 1, 1)
end_date = datetime(2024, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

data = []
for dt in date_range:
    dt_date = dt.to_pydatetime()
    
    # MedicalEventDateKey: yyyyMMdd as integer
    key = int(dt_date.strftime("%Y%m%d"))
    
    # MedicalEventDateDate: formatted as yyyy-mm-dd 00:00:00 (change to mm-dd-yyyy if needed)
    # The specification says mm-dd-yyyy format, so we convert accordingly.
    med_date_date = dt_date.strftime("%m-%d-%Y 00:00:00")
    
    # Determine quarter and first day of quarter
    quarter = ((dt_date.month - 1) // 3) + 1
    first_month_of_quarter = (quarter - 1) * 3 + 1
    first_day_of_quarter = datetime(dt_date.year, first_month_of_quarter, 1)
    day_in_quarter = (dt_date - first_day_of_quarter).days + 1
    
    # Day names and day of month
    day_name = dt_date.strftime("%A")
    day_abbr = dt_date.strftime("%a")
    day_of_month = dt_date.day
    
    # Numeric day of week (1=Monday, 7=Sunday). Python's weekday(): Monday=0 ... Sunday=6
    dow = dt_date.weekday()
    day_of_week = dow + 1  # Now Monday=1, Sunday=7
    
    # Occurrence count of that weekday in the month
    day_of_week_in_month = sum(1 for d in range(1, dt_date.day + 1)
                               if datetime(dt_date.year, dt_date.month, d).weekday() == dow)
    
    # Day of year
    day_of_year = dt_date.timetuple().tm_yday
    
    # First day of month, quarter, and year in specified format (mm-dd-yyyy 00:00:00)
    first_day_of_month = dt_date.replace(day=1).strftime("%m-%d-%Y 00:00:00")
    first_day_of_quarter_str = first_day_of_quarter.strftime("%m-%d-%Y 00:00:00")
    first_day_of_year = datetime(dt_date.year, 1, 1).strftime("%m-%d-%Y 00:00:00")
    
    # Determine holiday (simple rules for three common holidays)
    holiday = ""
    if dt_date.month == 1 and dt_date.day == 1:
        holiday = "New Year's Day"
    elif dt_date.month == 7 and dt_date.day == 4:
        holiday = "Independence Day"
    elif dt_date.month == 12 and dt_date.day == 25:
        holiday = "Christmas Day"
    
    is_holiday = "TRUE" if holiday else "FALSE"
    is_weekday = "TRUE" if dt_date.weekday() < 5 else "FALSE"
    is_weekend = "TRUE" if dt_date.weekday() >= 5 else "FALSE"
    
    # Last day of quarter
    last_month_of_quarter = first_month_of_quarter + 2
    last_day_q = calendar.monthrange(dt_date.year, last_month_of_quarter)[1]
    last_day_of_quarter = datetime(dt_date.year, last_month_of_quarter, last_day_q).strftime("%m-%d-%Y 00:00:00")
    
    # Last day of month and year
    last_day_m = calendar.monthrange(dt_date.year, dt_date.month)[1]
    last_day_of_month = dt_date.replace(day=last_day_m).strftime("%m-%d-%Y 00:00:00")
    last_day_of_year = datetime(dt_date.year, 12, 31).strftime("%m-%d-%Y 00:00:00")
    
    # Month details
    month = dt_date.month
    month_abbr = dt_date.strftime("%b")
    month_name = dt_date.strftime("%B")
    month_of_quarter = dt_date.month - first_month_of_quarter + 1  # as string below
    
    # Quarter details
    quarter_num = quarter
    quarter_name = {1: "First", 2: "Second", 3: "Third", 4: "Fourth"}[quarter_num]
    quarter_short_name = f"Q{quarter_num}"
    
    # Week of month, quarter, and year
    week_of_month = (dt_date.day - 1) // 7 + 1
    week_of_quarter = ((dt_date - first_day_of_quarter).days) // 7 + 1
    week_of_year = dt_date.isocalendar()[1]
    
    # YYYYMM formatted as yyyy/MM
    yyyy_mm = dt_date.strftime("%Y/%m")
    
    year = dt_date.year
    year_and_quarter = f"{year}/Q{quarter_num}"
    year_month = f"{year}/{dt_date.strftime('%b')}"
    year_name = f"CY {year}"
    
    # Fiscal calculations (assume fiscal year starts on October 1)
    if dt_date.month < 10:
        fiscal_start = datetime(dt_date.year - 1, 10, 1)
    else:
        fiscal_start = datetime(dt_date.year, 10, 1)
    fiscal_start_str = fiscal_start.strftime("%m-%d-%Y 00:00:00")
    
    # Fiscal Date Key: we can reuse key (or compute similarly)
    fiscal_date_key = key  # as string in our schema
    fiscal_day_of_year = (dt_date - fiscal_start).days + 1
    # Fiscal month: October becomes month 1, November 2, etc.
    fiscal_month = dt_date.month - 9 if dt_date.month >= 10 else dt_date.month + 3
    fiscal_quarter = ((fiscal_month - 1) // 3) + 1
    fiscal_quarter_name = f"Q{fiscal_quarter}"
    fiscal_week_of_year = ((dt_date - fiscal_start).days) // 7 + 1
    fiscal_year_val = dt_date.year if dt_date.month < 10 else dt_date.year + 1
    fiscal_year = f"FY{fiscal_year_val}"
    
    is_first_day_fiscal = "TRUE" if dt_date.date() == fiscal_start.date() else "FALSE"
    fiscal_year_end = datetime(fiscal_year_val, 9, 30)
    is_last_day_fiscal = "TRUE" if dt_date.date() == fiscal_year_end.date() else "FALSE"
    last_day_fiscal = fiscal_year_end.strftime("%m-%d-%Y 00:00:00")
    
    row = {
        "MedicalEventDateKey": key,
        "MedicalEventDateDate": med_date_date,
        "MedicalEventDateDayInQuarter": str(day_in_quarter),
        "MedicalEventDateDayName": day_name,
        "MedicalEventDateDayNameAbbrevation": day_abbr,
        "MedicalEventDateDayOfMonth": str(day_of_month),
        "MedicalEventDateDayOfWeek": str(day_of_week),
        "MedicalEventDateDayOfWeekInMonth": str(day_of_week_in_month),
        "MedicalEventDateDayOfYear": str(day_of_year),
        "MedicalEventDateFirstDayOfMonth": first_day_of_month,
        "MedicalEventDateFirstDayOfQuarter": first_day_of_quarter_str,
        "MedicalEventDateFirstDayofYear": first_day_of_year,
        "MedicalEventDateHoliday": holiday,
        "MedicalEventDateIsHoliday": is_holiday,
        "MedicalEventDateIsWeekday": is_weekday,
        "MedicalEventDateIsWeekend": is_weekend,
        "MedicalEventDateLastDayOfQuarter": last_day_of_quarter,
        "MedicalEventDateLastDayofMonth": last_day_of_month,
        "MedicalEventDateLastDayofYear": last_day_of_year,
        "MedicalEventDateMonth": str(month),
        "MedicalEventDateMonthAbbrevation": month_abbr,
        "MedicalEventDateMonthName": month_name,
        "MedicalEventDateMonthOfQuarter": str(month_of_quarter),
        "MedicalEventDateQuarter": str(quarter_num),
        "MedicalEventDateQuarterName": quarter_name,
        "MedicalEventDateQuarterShortName": quarter_short_name,
        "MedicalEventDateWeekOfMonth": str(week_of_month),
        "MedicalEventDateWeekOfQuarter": str(week_of_quarter),
        "MedicalEventDateWeekOfYear": str(week_of_year),
        "MedicalEventDateYYYYMM": yyyy_mm,
        "MedicalEventDateYear": str(year),
        "MedicalEventDateYearAndQuarter": year_and_quarter,
        "MedicalEventDateYearMonth": year_month,
        "MedicalEventDateYearName": year_name,
        "MedicalEventDateFirstDayOfFiscalYear": fiscal_start_str,
        "MedicalEventDateFiscalDateKey": str(fiscal_date_key),
        "MedicalEventDateFiscalDayOfYear": str(fiscal_day_of_year),
        "MedicalEventDateFiscalMonth": str(fiscal_month),
        "MedicalEventDateFiscalQuarter": str(fiscal_quarter),
        "MedicalEventDateFiscalQuarterName": fiscal_quarter_name,
        "MedicalEventDateFiscalWeekOfYear": str(fiscal_week_of_year),
        "MedicalEventDateFiscalYear": fiscal_year,
        "MedicalEventDateIsFirstDayOfFiscalYear": is_first_day_fiscal,
        "MedicalEventDateIsLastOfFiscalYear": is_last_day_fiscal,
        "MedicalEventDateLastDayOfFiscalYear": last_day_fiscal
    }
    data.append(row)

# Convert the list of dictionaries into a DataFrame.
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file.
csv_filename = "Dim_MedicalEventDate.csv"
df.to_csv(csv_filename, index=False)
print(f"CSV file '{csv_filename}' created with {len(df)} records.")

# Create (or connect to) the SQLite database and store the data.
db_filename = "medical_events.db"
conn = sqlite3.connect(db_filename)
df.to_sql("Dim_MedicalEventDate", conn, if_exists='replace', index=False)
conn.commit()
conn.close()
print(f"Data stored in SQLite database '{db_filename}' in table 'Dim_MedicalEventDate'.")


CSV file 'Dim_MedicalEventDate.csv' created with 1461 records.
Data stored in SQLite database 'medical_events.db' in table 'Dim_MedicalEventDate'.


In [13]:
import pandas as pd
import sqlite3
import calendar
from datetime import datetime, timedelta

# Define the date range: from 2021-01-01 to 2024-12-31
start_date = datetime(2021, 1, 1)
end_date = datetime(2024, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

data = []
for dt in date_range:
    dt_date = dt.to_pydatetime()
    
    # ReportDateKey: yyyyMMdd as integer
    key = int(dt_date.strftime("%Y%m%d"))
    
    # ReportDateDate: formatted as mm-dd-yyyy 00:00:00
    report_date_date = dt_date.strftime("%m-%d-%Y 00:00:00")
    
    # Determine quarter and first day of quarter
    quarter = ((dt_date.month - 1) // 3) + 1
    first_month_of_quarter = (quarter - 1) * 3 + 1
    first_day_of_quarter = datetime(dt_date.year, first_month_of_quarter, 1)
    day_in_quarter = (dt_date - first_day_of_quarter).days + 1
    
    # Day names and day of month
    day_name = dt_date.strftime("%A")
    day_abbr = dt_date.strftime("%a")
    day_of_month = dt_date.day
    
    # Numeric day of week (1=Monday, 7=Sunday); Python: Monday=0 ... Sunday=6
    dow = dt_date.weekday()
    day_of_week = dow + 1
    
    # Occurrence count of that weekday in the month
    day_of_week_in_month = sum(1 for d in range(1, dt_date.day + 1)
                               if datetime(dt_date.year, dt_date.month, d).weekday() == dow)
    
    # Day of year
    day_of_year = dt_date.timetuple().tm_yday
    
    # First day values for month, quarter, and year (formatted as mm-dd-yyyy 00:00:00)
    first_day_of_month = dt_date.replace(day=1).strftime("%m-%d-%Y 00:00:00")
    first_day_of_quarter_str = first_day_of_quarter.strftime("%m-%d-%Y 00:00:00")
    first_day_of_year = datetime(dt_date.year, 1, 1).strftime("%m-%d-%Y 00:00:00")
    
    # Determine holiday (simple rules for three common holidays)
    holiday = ""
    if dt_date.month == 1 and dt_date.day == 1:
        holiday = "New Year's Day"
    elif dt_date.month == 7 and dt_date.day == 4:
        holiday = "Independence Day"
    elif dt_date.month == 12 and dt_date.day == 25:
        holiday = "Christmas Day"
    
    is_holiday = "TRUE" if holiday else "FALSE"
    is_weekday = "TRUE" if dt_date.weekday() < 5 else "FALSE"
    is_weekend = "TRUE" if dt_date.weekday() >= 5 else "FALSE"
    
    # Last day of quarter
    last_month_of_quarter = first_month_of_quarter + 2
    last_day_q = calendar.monthrange(dt_date.year, last_month_of_quarter)[1]
    last_day_of_quarter = datetime(dt_date.year, last_month_of_quarter, last_day_q).strftime("%m-%d-%Y 00:00:00")
    
    # Last day of month and year
    last_day_m = calendar.monthrange(dt_date.year, dt_date.month)[1]
    last_day_of_month = dt_date.replace(day=last_day_m).strftime("%m-%d-%Y 00:00:00")
    last_day_of_year = datetime(dt_date.year, 12, 31).strftime("%m-%d-%Y 00:00:00")
    
    # Month details
    month = dt_date.month
    month_abbr = dt_date.strftime("%b")
    month_name = dt_date.strftime("%B")
    month_of_quarter = dt_date.month - first_month_of_quarter + 1
    
    # Quarter details
    quarter_num = quarter
    quarter_name = {1: "First", 2: "Second", 3: "Third", 4: "Fourth"}[quarter_num]
    quarter_short_name = f"Q{quarter_num}"
    
    # Week numbers
    week_of_month = (dt_date.day - 1) // 7 + 1
    week_of_quarter = ((dt_date - first_day_of_quarter).days) // 7 + 1
    week_of_year = dt_date.isocalendar()[1]
    
    # YYYYMM formatted as yyyy/MM
    yyyy_mm = dt_date.strftime("%Y/%m")
    
    year = dt_date.year
    year_and_quarter = f"{year}/Q{quarter_num}"
    year_month = f"{year}/{dt_date.strftime('%b')}"
    year_name = f"CY {year}"
    
    # Fiscal calculations (assume fiscal year starts on October 1)
    if dt_date.month < 10:
        fiscal_start = datetime(dt_date.year - 1, 10, 1)
    else:
        fiscal_start = datetime(dt_date.year, 10, 1)
    fiscal_start_str = fiscal_start.strftime("%m-%d-%Y 00:00:00")
    
    # Fiscal Date Key (reuse key) and other fiscal values
    fiscal_date_key = key
    fiscal_day_of_year = (dt_date - fiscal_start).days + 1
    fiscal_month = dt_date.month - 9 if dt_date.month >= 10 else dt_date.month + 3
    fiscal_quarter = ((fiscal_month - 1) // 3) + 1
    fiscal_quarter_name = f"Q{fiscal_quarter}"
    fiscal_week_of_year = ((dt_date - fiscal_start).days) // 7 + 1
    fiscal_year_val = dt_date.year if dt_date.month < 10 else dt_date.year + 1
    fiscal_year = f"FY{fiscal_year_val}"
    
    is_first_day_fiscal = "TRUE" if dt_date.date() == fiscal_start.date() else "FALSE"
    fiscal_year_end = datetime(fiscal_year_val, 9, 30)
    is_last_day_fiscal = "TRUE" if dt_date.date() == fiscal_year_end.date() else "FALSE"
    last_day_fiscal = fiscal_year_end.strftime("%m-%d-%Y 00:00:00")
    
    row = {
        "ReportDateKey": key,
        "ReportDateDate": report_date_date,
        "ReportDateDayInQuarter": str(day_in_quarter),
        "ReportDateDayName": day_name,
        "ReportDateDayNameAbbrevation": day_abbr,
        "ReportDateDayOfMonth": str(day_of_month),
        "ReportDateDayOfWeek": str(day_of_week),
        "ReportDateDayOfWeekInMonth": str(day_of_week_in_month),
        "ReportDateDayOfYear": str(day_of_year),
        "ReportDateFirstDayOfMonth": first_day_of_month,
        "ReportDateFirstDayOfQuarter": first_day_of_quarter_str,
        "ReportDateFirstDayofYear": first_day_of_year,
        "ReportDateHoliday": holiday,
        "ReportDateIsHoliday": is_holiday,
        "ReportDateIsWeekday": is_weekday,
        "ReportDateIsWeekend": is_weekend,
        "ReportDateLastDayOfQuarter": last_day_of_quarter,
        "ReportDateLastDayofMonth": last_day_of_month,
        "ReportDateLastDayofYear": last_day_of_year,
        "ReportDateMonth": str(month),
        "ReportDateMonthAbbrevation": month_abbr,
        "ReportDateMonthName": month_name,
        "ReportDateMonthOfQuarter": str(month_of_quarter),
        "ReportDateQuarter": str(quarter_num),
        "ReportDateQuarterName": quarter_name,
        "ReportDateQuarterShortName": quarter_short_name,
        "ReportDateWeekOfMonth": str(week_of_month),
        "ReportDateWeekOfQuarter": str(week_of_quarter),
        "ReportDateWeekOfYear": str(week_of_year),
        "ReportDateYYYYMM": yyyy_mm,
        "ReportDateYear": str(year),
        "ReportDateYearAndQuarter": year_and_quarter,
        "ReportDateYearMonth": year_month,
        "ReportDateYearName": year_name,
        "ReportDateFirstDayOfFiscalYear": fiscal_start_str,
        "ReportDateFiscalDateKey": str(fiscal_date_key),
        "ReportDateFiscalDayOfYear": str(fiscal_day_of_year),
        "ReportDateFiscalMonth": str(fiscal_month),
        "ReportDateFiscalQuarter": str(fiscal_quarter),
        "ReportDateFiscalQuarterName": fiscal_quarter_name,
        "ReportDateFiscalWeekOfYear": str(fiscal_week_of_year),
        "ReportDateFiscalYear": fiscal_year,
        "ReportDateIsFirstDayOfFiscalYear": is_first_day_fiscal,
        "ReportDateIsLastOfFiscalYear": is_last_day_fiscal,
        "ReportDateLastDayOfFiscalYear": last_day_fiscal
    }
    data.append(row)

# Convert the list of dictionaries into a DataFrame.
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file.
csv_filename = "Dim_ReportDate.csv"
df.to_csv(csv_filename, index=False)
print(f"CSV file '{csv_filename}' created with {len(df)} records.")

# Create (or connect to) the SQLite database and store the data.
db_filename = "medical_events.db"
conn = sqlite3.connect(db_filename)
df.to_sql("Dim_ReportDate", conn, if_exists='replace', index=False)
conn.commit()
conn.close()
print(f"Data stored in SQLite database '{db_filename}' in table 'Dim_ReportDate'.")


CSV file 'Dim_ReportDate.csv' created with 1461 records.
Data stored in SQLite database 'medical_events.db' in table 'Dim_ReportDate'.


In [25]:
import pandas as pd
import sqlite3
import random
from datetime import datetime, timedelta

# Define date range for report and medical event dates.
date_start = datetime(2021, 1, 1)
date_end = datetime(2024, 12, 31)

def random_date_key(start, end):
    """Generate a random date between start and end, formatted as yyyyMMdd (integer)."""
    delta = end - start
    random_days = random.randint(0, delta.days)
    date_val = start + timedelta(days=random_days)
    return int(date_val.strftime("%Y%m%d"))

def random_time():
    """Generate a random time in h:mm:ss AM/PM format."""
    hour = random.randint(1, 12)
    minute = random.randint(0, 59)
    second = random.randint(0, 59)
    am_pm = random.choice(["AM", "PM"])
    return f"{hour}:{minute:02d}:{second:02d} {am_pm}"

# Define lists for random selection for text fields.
detail_options = [
    "COLONY COUNT: 100,000+ GRAM-POSITIVE COCCI IN CHAINS",
    "COLONY COUNT: 50,000 MIXED FLORA - THREE OR MORE SPECIES PRESENT",
    "ISOLATION OF THREE OR MORE DIFFERENT BACTERIA. PLEASE REPEAT IF CLINICALLY INDICATED",
    "LABORATORY CONFIRMATION OF BACTERIAL PATHOGEN",
    "SIGNIFICANT ELEVATION IN WHITE BLOOD CELL COUNT"
]

prescription_options = [
    "Molnupiravir Oral Capsule 200 MG (Aug 22, 2024 - Aug 27, 2024)",
    "Flomax Capsule 0.4 MG (Dec 07, 2024 - Indefinite)",
    "Atorvastatin Tablet 20 MG (Jan 01, 2025 - Indefinite)",
    "Metformin Tablet 500 MG (Feb 15, 2025 - Indefinite)"
]

room_options = [
    "cafeteria", "activity room", "E110", "E120", "Nursing Station", "Lobby"
]

num_records = 200
fact_data = []

for i in range(1, num_records + 1):
    record = {
        "ResidentKey": random.randint(1, 500),  # from Dim_Resident
        "FacilityKey": random.randint(1, 5),     # from Dim_Facility
        "UnitKey": random.randint(1, 10),         # from Dim_Unit
        "LocationKey": random.randint(1, 5),     # from Dim_EHRLocation (or similar)
        "MedicalEventTypeKey": random.randint(1, 26),  # from Dim_MedicalEventType
        "MedicalEventSeverityKey": random.randint(1, 20),  # from Dim_MedicalEventSeverity
        "EventStatusKey": random.randint(1, 8),   # from Dim_EventStatus
        "ReportDateKey": random_date_key(date_start, date_end),  # from Dim_ReportDate
        "MedicalEventDateKey": random_date_key(date_start, date_end),  # from Dim_MedicalEventDate
        "EventId": str(i),
        "MedicalEventFactId": "MEF" + str(i),
        "MedicalEventDetail": random.choice(detail_options),
        "Evaluation": random.choice(["Positive", "Negative", "Inconclusive"]),
        "Etiology": random.choice(["On Admission", "In House", "Community-Acquired"]),
        "Prescription": random.choice(prescription_options),
        "MedicalEventRoom": random.choice(room_options),
        "MedicalEventTime": random_time(),
        "InjuryFlag": random.choice(["Y", "N"]),
        "WoundFlag": random.choice(["Y", "N"]),
        "MedicationErrorFlag": random.choice(["Y", "N"]),
        "NatureOfInjury": random.choice(["wound re-opened", "ulcer", "none", "abrasion"]),
        "DegreeOfInjury": random.choice([
            "Moderate Injury, Treatment Required", 
            "Mild Injury, First Aid Required", 
            "Severe Injury, Hospitalization Required", 
            "No Injury"
        ]),
        "SafetyPrecautionTaken": random.choice([
            "wheelchair brakes on", "walker available", "none", "bed rails secured"
        ]),
        "Organism": random.choice([
            "ENTEROCOCCUS FAECIUM", "PROTEUS MIRABILIS", "ENTEROCOCCUS FAECALIS", 
            "STAPHYLOCOCCUS AUREUS", "Escherichia Coli", "Pseudomonas aeruginosa", 
            "Klebsiella pneumoniae", "NONE"
        ])
    }
    fact_data.append(record)

# Create a DataFrame from the fact records.
df_fact = pd.DataFrame(fact_data)

# Save the DataFrame to a CSV file.
csv_filename = "Fact_MedicalEvent.csv"
df_fact.to_csv(csv_filename, index=False)
print(f"CSV file '{csv_filename}' created with {num_records} records.")

# Connect to the SQLite database and store the fact table.
db_filename = "medical_events.db"
conn = sqlite3.connect(db_filename)
df_fact.to_sql("Fact_MedicalEvent", conn, if_exists='replace', index=False)
conn.commit()
conn.close()
print(f"Data stored in SQLite database '{db_filename}' in table 'Fact_MedicalEvent'.")


CSV file 'Fact_MedicalEvent.csv' created with 200 records.
Data stored in SQLite database 'medical_events.db' in table 'Fact_MedicalEvent'.


In [18]:
import pandas as pd
import sqlite3
import random
from datetime import datetime, timedelta
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# ----- Your existing fact table generation code (simplified) -----
# Assuming you have already generated your fact table DataFrame as df_fact
# For example:
df_fact = pd.read_csv("Fact_MedicalEvent.csv")
# (Or use your generation code provided earlier.)

# For demonstration, let's assume df_fact exists. (Replace this with your actual df_fact.)
# --------------------------------------------------------------

# Initialize the LLM (ensure you have your config and API key set correctly)
llm = AzureChatOpenAI(
    temperature=0.1,
    deployment_name="az-gpt_35_model",
    model_name="gpt-3.5-turbo",
    azure_endpoint="https://az-openai-document-question-answer-service.openai.azure.com/",
    openai_api_version="2023-05-15",
    openai_api_key="5d24331966b648738e5003caad552df8"            
)

# Define the prompt for generating clinical notes.
notes_gen_prompt = ("""
    "You are a medical note generator. Based on the following medical event data, "
    "generate a detailed clinical note that a nurse might record in the patient's chart. "
    "Incorporate relevant details such as DegreeOfInjury, NatureOfInjury, SafetyPrecautionTaken, "
    "Prescription, MedicalEventDetail, and Evaluation into a coherent narrative. "
    "Do not simply list the values; produce a natural language note."
    Use Below Format : 
   1.  Upon observation the nurse noted dried blood under the left foot 5th toenail, 0.2x0.2cm skin tear on left 4th toe. Right shin with superficial skin tear 0.1x0.1x<0.1cm. 
   Resident stated, "he bumped himself in the gym at 2pm this afternoon, and that he scratched his leg".
2. Resident found to have a skin tear to the right outer ankle.  
called by OT to assessed skin tear to left FA, skin tear measured 5.5cm x 4cm x0.1cm. half moon shape skin flap scant bleeding noted. 
and Vera denies any pain. Per Loren it happened during transfer from toilet to chair. 
she and the CNA assisted her during the transfer.  MD notified and received order to apply steri strip xeroform cover abd. pad wrap with kling and wound consult. daughter Lyn aware as well.

3. Discoloration with a skin tear measuring 1cm x 1.4cm to right lower leg. 
4. Resident sustained a sheered skin tear to right lower leg during transfer. 
5. Skin tear might have been caused by resident's action while sitting on the wheelchair. Site of skin tear was the same level as where the leg rest were in place.
6. esident was sent out to the ER for agitation during the night, came back with bruise and skin tear. Prior going to the ER his skin was intact as per last weekly skin observation on 3/21/24.
,resident reported to the care attendant a self-inflicted wound to the back of his hand. He  told care partner that he caught his hand-  back of his hand, on the side of the bedside table on front of him, a skin tear measuring 1cm x 1 cm was noted in between his 2nd and 3rd knuckles. Skin tear protocol initiated.
7. This writer with CNA  was transferring resident with Hoyer lift.  Instructed resident to hug himself , resident  then lifted his arms to hold the Hoyer, accidentally hit his right hand on the mechanical lift (Hoyer) machine and sustained skin tear. Scant amount of bleeding noted. Skin tear measured :5cm x 3.5cm. Tx given. Megan  APN notified.  Ordered to apply Xeroform daily until heal. Daughter Diane visited during dinner time and notified. 
    """
)
prompt = ChatPromptTemplate.from_messages([
    ("system", notes_gen_prompt),
    ("human", "Medical event data:\n{data_from_fact}")
])

# Define a function that takes a fact record (row) and returns a generated ClinicalNotes string.
def generate_clinical_notes(row):
    # Construct an input string from the key columns.
    data_from_fact = (
        f"DegreeOfInjury: {row['DegreeOfInjury']}\n"
        f"NatureOfInjury: {row['NatureOfInjury']}\n"
        f"SafetyPrecautionTaken: {row['SafetyPrecautionTaken']}\n"
        f"Prescription: {row['Prescription']}\n"
        f"MedicalEventDetail: {row['MedicalEventDetail']}\n"
        f"Evaluation: {row['Evaluation']}\n"
        f"Etiology: {row.get('Etiology', 'N/A')}\n"
        f"Organism: {row.get('Organism', 'N/A')}"
    )
    # Format the prompt.
    prompt_input = prompt.format_prompt(data_from_fact=data_from_fact)
    # Invoke the LLM.
    response = llm.invoke(prompt_input)
    return response.content.strip()

# For demonstration, process a small sample first.
# Uncomment the next two lines to process only the first 10 rows:
# df_sample = df_fact.head(10).copy()
# df_sample["ClinicalNotes"] = df_sample.apply(generate_clinical_notes, axis=1); print(df_sample[["MedicalEventDetail", "ClinicalNotes"]])

# To update the full fact table with clinical notes, apply the function row-wise.
df_fact["ClinicalNotes"] = df_fact.apply(generate_clinical_notes, axis=1)

# Save the updated fact table to a new CSV file.
csv_filename_updated = "Fact_MedicalEvent_WithNotes.csv"
df_fact.to_csv(csv_filename_updated, index=False)
print(f"CSV file '{csv_filename_updated}' created with ClinicalNotes column.")

# Update the SQLite database with the new fact table.
db_filename = "medical_events.db"
conn = sqlite3.connect(db_filename)
df_fact.to_sql("Fact_MedicalEvent", conn, if_exists='replace', index=False)
conn.commit()
conn.close()
print(f"Updated Fact_MedicalEvent table stored in SQLite database '{db_filename}'.")


CSV file 'Fact_MedicalEvent_WithNotes.csv' created with ClinicalNotes column.
Updated Fact_MedicalEvent table stored in SQLite database 'medical_events.db'.


In [28]:
csv_filename_updated = "Fact_MedicalEvent_WithNotes.csv"
df_fact = pd.read_csv(csv_filename_updated)
print(f"CSV file '{csv_filename_updated}' created with ClinicalNotes column.")

# Update the SQLite database with the new fact table.
db_filename = "medical_events.db"
conn = sqlite3.connect(db_filename)
df_fact.to_sql("Fact_MedicalEvent", conn, if_exists='replace', index=False)
conn.commit()
conn.close()
print(f"Updated Fact_MedicalEvent table stored in SQLite database '{db_filename}'.")

CSV file 'Fact_MedicalEvent_WithNotes.csv' created with ClinicalNotes column.
Updated Fact_MedicalEvent table stored in SQLite database 'medical_events.db'.


In [20]:
conn = sqlite3.connect(db_filename)

# Debug: list available tables in the database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
print("Available tables in DB:", tables)

Available tables in DB: [('Dim_CensusRoomType',), ('Dim_CensusStatus',), ('Dim_CensusDate',), ('Dim_CensusResident',), ('Dim_CensusFacility',), ('Dim_CensusLocation',), ('Dim_CensusUnit',), ('Fact_Census',)]
